In [1]:
import pyranges as pr
import pandas as pd
from cerberus.cerberus import *

## end aggregation

In [54]:
beds = ['tests/files/Canx_1_tss.bed', 'tests/files/Canx_2_tss.bed', 'tests/files/Canx_ccre_pels.bed']
sources = ['v1', 'v2', 'pels_ccre']
add_ends = [True, True, True]
buffer = 300
mode = 'tss'

df = pd.DataFrame()
i = 0
for bed_fname, source, add in zip(beds, sources, add_ends):

    bed = read_bed(bed_fname, mode)
    bed = bed.df 
    bed['source'] = source
    bed = pr.PyRanges(bed)
    
    # first bed; just use ends
    if len(df.index) == 0:
        df = bed.df
        
    # construct the join call based on what information is present in the new bed
    if 'Strand' in bed.columns:
        pass
    if 'gene_id' in bed.columns:
        pass

    # if we're adding new ends, make sure to do a right merge and renumber

            
        
    i +=1
    if i ==2: break

In [55]:
df = pr.PyRanges(df)

In [56]:
df

,Chromosome,Start,End,Name,Score,Strand,gene_id,tss,source
0,chr11,50298268,50298369,ENSMUSG00000020368_3,.,-,ENSMUSG00000020368,3,v1
1,chr11,50300939,50301040,ENSMUSG00000020368_2,.,-,ENSMUSG00000020368,2,v1
2,chr11,50325622,50325723,ENSMUSG00000020368_1,.,-,ENSMUSG00000020368,1,v1


In [57]:
bed

,Chromosome,Start,End,Name,Score,Strand,gene_id,tss,source
0,chr11,50299327,50299428,ENSMUSG00000020368_2,.,-,ENSMUSG00000020368,2,v2
1,chr11,50300939,50301040,ENSMUSG00000020368_1,.,-,ENSMUSG00000020368,1,v2


In [66]:
# def merge_ends(old_ends, new_ends, strand, gene_id, how):
#     """
#     Parameters:
#         strand (bool)
#         gene_id (bool)
#         how (str): {'left', 'right'}
#     """
    
#     # if we have strand information, enforce that
#     if strand:
#         temp = df.join(bed,
#                        strandedness='same',
#                        suffix='_new',
#                        slack=buffer,
#                        how=how)
#     else:
#         temp = df.join(bed,
#                        strandedness=False,
#                        suffix='_new', 
#                        slack=buffer,
#                        how=how)
    
#     if gene_id:
#         temp = temp.loc[(temp.gene_id == temp.gene_id_new)|(temp.gene_id_new == '-1')]
    
#     return temp

In [271]:
def get_test_agg_2_bed():
    # example should have 
    # - entries that overlap
    # - entries that don't overlap but are within a certain distance
    # - entries that are unique to either
    # - entries that overlap but aren't using the same gene_id

    # beds that don't have strand
    # beds that don't have gid

    buffer = 20

    chroms = [1,1,1,1]
    starts = [1, 200, 100, 300]
    ends = [15, 250, 110,340]
    strands = ['+','+','+', '+']
    gids = ['gene_1', 'gene_1', 'gene_1', 'gene_1']
    source = 'v1'
    bed1 = pr.PyRanges(chromosomes=chroms, starts=starts, ends=ends).as_df()
    bed1['Strand'] = strands
    bed1['gene_id'] = gids
    bed1['source'] = source
    bed1 = pr.PyRanges(df=bed1)

    chroms = [1,1,1,1]
    starts = [5, 120, 500, 200]
    ends = [10, 140, 550, 250]
    strands=['+','+','+','+']
    gids = ['gene_1', 'gene_1', 'gene_1', 'gene_2']
    temp_ids = [i for i in range(len(chroms))]
    source = 'v2'
    bed2 = pyranges.PyRanges(chromosomes=chroms, starts=starts, ends=ends).as_df()
    bed2['gene_id'] = gids
    bed2['Strand'] = strands
    bed2['source'] = source
    bed2['id'] = temp_ids
    bed2 = pyranges.PyRanges(df=bed2)
    
    return bed1, bed2

In [278]:
def get_ctrl_ends(add=True):
    df = pd.DataFrame()
    rows = [1,200,100,300, 200,500]
    df['Start'] = rows
    rows = ['1' for i in range(len(df.index))]
    df['Chromosome'] = rows
    rows = [15,250,110,340, 250,550]
    df['End'] = rows
    rows = ['gene_1','gene_1','gene_1','gene_1', 'gene_2','gene_1']
    df['gene_id'] = rows
    rows = ['+' for i in range(len(df.index))]
    df['Strand'] = rows
    rows = ['v1,v2','v1','v1,v2','v1', 'v2','v2']
    df['source'] = rows
    
    # convert a few dtypes
    df['Strand'] = df['Strand'].astype('category')
    df['Chromosome'] = df['Chromosome'].astype('category')
    
    # if we're not adding new ends
    if not add:
        df = df.loc[df.source != 'v2']
    
    return df

In [288]:
def agg_2_ends(bed1, bed2, buffer, add_ends):
    """
    Parameters:
        bed1 (pyranges PyRanges): Bed PyRanges object for existing ends
        bed2 (pyranges PyRanges): Bed PyRanges object for new ends
        buffer (int): Maximum allowable distance between ends in bed1 and bed2 
            to call them the same end
        add_ends (bool): Whether to initialize new regions from bed2
    """
    
    temp_joined = bed1.join(bed2, 
        strandedness='same',
        suffix='_new',
        slack=buffer,
        how='left')


    temp_joined = temp_joined.df
    # print(temp_joined)
    df = pd.DataFrame()

    ### old ends ###

    # situation 1: ends match across the datasets in coord and gene id
    temp = temp_joined.loc[temp_joined.gene_id == temp_joined.gene_id_new].copy(deep=True)
    temp.source = temp.source+','+temp.source_new
    # print(temp)
    df = pd.concat([df, temp])
    # print(df)

    # situation 2: ends are only in the first dataset
    temp = temp_joined.loc[(temp_joined.gene_id_new == '-1')|(temp_joined.gene_id!=temp_joined.gene_id_new)].copy(deep=True)
    # print('here')
    # print(temp)
    df = pd.concat([df, temp])
    # print(df)
    # print()

    ### new ends, only add if we're allowing them to be independent
    ### end support
    if add_ends:
        drop_cols = ['Start', 'End', 'Strand', 'gene_id', 'source']
        m = {'Start_new': 'Start', 
             'End_new': 'End',
             'gene_id_new': 'gene_id',
             'Strand_new': 'Strand',
             'source_new': 'source'}

        # situation 3: the ends overlapped, but the gene ids didn't match
        temp = temp_joined.loc[(temp_joined.gene_id!=temp_joined.gene_id_new)&(temp_joined.gene_id_new!='-1')].copy(deep=True)
        temp.drop(drop_cols, axis=1, inplace=True)
        temp.rename(m, axis=1, inplace=True)
        # print('hewwo')
        # print(temp.head())
        df = pd.concat([df, temp])
        # print(df)

        # situation 4: the ends are brand new and didn't overlap at all in the existing ends
        bed2 = bed2.df
        # print(bed2.id.dtypes)
        # print(temp.id.dtypes)
        # print(bed2.id.tolist())
        # print(df.id.tolist())
        inds = list(set(bed2.id.tolist())-set(df.id.tolist()))
        # print(inds)
        temp = bed2.loc[inds]
        # print(temp)
        df = pd.concat([df, temp])

    # drop unnecessary columns
    drop_cols = ['id', 'Start_new', 'End_new', 'gene_id_new', 'Strand_new', 'source_new']
    df.drop(drop_cols, axis=1, inplace=True)
    
    return df

In [299]:
def test_agg_2_ends():
    """
    Test agg_2_ends w/ end adding
    """
    tests = [True, False]
    for add_ends in tests:
        
        buffer = 20
        sort_cols = ['Chromosome', 'Strand', 'gene_id', 'Start', 'End']
        bed1, bed2 = get_test_agg_2_bed()

        df = agg_2_ends(bed1, bed2,
                        buffer=buffer,
                        add_ends=add_ends)
        df = df.sort_values(by=sort_cols)
        df.reset_index(drop=True, inplace=True)

        ctrl = get_ctrl_ends(add=add_ends)
        ctrl = ctrl.sort_values(by=sort_cols)
        ctrl.reset_index(drop=True, inplace=True)

        pd.testing.assert_frame_equal(ctrl, df, check_like=True)
        assert len(ctrl.index) == len(df.index)

In [300]:
test_agg_2_ends()

In [ ]:
ctrl

,Start,Chromosome,End,gene_id,Strand,source
0,1,1,15,gene_1,+,"v1,v2"
1,100,1,110,gene_1,+,"v1,v2"
2,200,1,250,gene_1,+,v1
3,300,1,340,gene_1,+,v1
4,500,1,550,gene_1,+,v2
5,200,1,250,gene_2,+,v2


In [ ]:
df

,Chromosome,Start,End,Strand,gene_id,source
0,1,1,15,+,gene_1,"v1,v2"
1,1,100,110,+,gene_1,"v1,v2"
2,1,200,250,+,gene_1,v1
3,1,300,340,+,gene_1,v1
4,1,500,550,+,gene_1,v2
5,1,200,250,+,gene_2,v2


In [ ]:
buffer = 20
add_ends = False
bed1, bed2 = get_test_agg_2_bed()

df = agg_2_ends(bed1, bed2,
                buffer=buffer,
                add_ends=add_ends)
df = df.sort_values(by=sort_cols)
df.reset_index(drop=True, inplace=True)

ctrl = get_ctrl_ends(add=add_ends)
ctrl = ctrl.sort_values(by=sort_cols)
ctrl.reset_index(drop=True, inplace=True)

pd.testing.assert_frame_equal(ctrl, df, check_like=True)
assert len(ctrl.index) == len(df.index)

In [ ]:
ctrl

,Start,Chromosome,End,gene_id,Strand,source
0,1,1,15,gene_1,+,"v1,v2"
1,100,1,110,gene_1,+,"v1,v2"
2,200,1,250,gene_1,+,v1
3,300,1,340,gene_1,+,v1


In [ ]:
df

,Chromosome,Start,End,Strand,gene_id,source
0,1,1,15,+,gene_1,"v1,v2"
1,1,100,110,+,gene_1,"v1,v2"
2,1,200,250,+,gene_1,v1
3,1,300,340,+,gene_1,v1


In [ ]:
temp_joined = bed1.join(bed2, 
        strandedness='same',
        suffix='_new',
        slack=buffer,
        how='left')


temp_joined = temp_joined.df
# print(temp_joined)
df = pd.DataFrame()

### old ends ###

# situation 1: ends match across the datasets in coord and gene id
temp = temp_joined.loc[temp_joined.gene_id == temp_joined.gene_id_new]
temp.source = temp.source+','+temp.source_new
# print(temp)
df = pd.concat([df, temp])
print(df)


# situation 2: ends are only in the first dataset
temp = temp_joined.loc[(temp_joined.gene_id_new == '-1')|(temp_joined.gene_id!=temp_joined.gene_id_new)]
# print('here')
# print(temp)
df = pd.concat([df, temp])
print(df)
print()

### new ends, only add if we're allowing them to be independent
### end support
if add:
    drop_cols = ['Start', 'End', 'Strand', 'gene_id', 'source']
    m = {'Start_new': 'Start', 
         'End_new': 'End',
         'gene_id_new': 'gene_id',
         'Strand_new': 'Strand',
         'source_new': 'source'}
    
    # situation 3: the ends overlapped, but the gene ids didn't match
    temp = temp_joined.loc[(temp_joined.gene_id!=temp_joined.gene_id_new)&(temp_joined.gene_id_new!='-1')].copy(deep=True)
    temp.drop(drop_cols, axis=1, inplace=True)
    temp.rename(m, axis=1, inplace=True)
    # print('hewwo')
    # print(temp.head())
    df = pd.concat([df, temp])
    # print(df)
    
    # situation 4: the ends are brand new and didn't overlap at all in the existing ends
    bed2 = bed2.df
    # print(bed2.id.dtypes)
    # print(temp.id.dtypes)
    # print(bed2.id.tolist())
    # print(df.id.tolist())
    inds = list(set(bed2.id.tolist())-set(df.id.tolist()))
    # print(inds)
    temp = bed2.loc[inds]
    # print(temp)
    df = pd.concat([df, temp])

# drop unnecessary columns
drop_cols = ['id', 'Start_new', 'End_new', 'gene_id_new', 'Strand_new', 'source_new']
df.drop(drop_cols, axis=1, inplace=True)

  Chromosome  Start  End Strand gene_id source  Start_new  End_new  \
0          1      1   15      +  gene_1  v1,v2          5       10   
2          1    100  110      +  gene_1  v1,v2        120      140   

  gene_id_new Strand_new source_new  id  
0      gene_1          +         v2   0  
2      gene_1          +         v2   1  
  Chromosome  Start  End Strand gene_id source  Start_new  End_new  \
0          1      1   15      +  gene_1  v1,v2          5       10   
2          1    100  110      +  gene_1  v1,v2        120      140   
1          1    200  250      +  gene_1     v1        200      250   
3          1    300  340      +  gene_1     v1         -1       -1   

  gene_id_new Strand_new source_new  id  
0      gene_1          +         v2   0  
2      gene_1          +         v2   1  
1      gene_2          +         v2   3  
3          -1          +         -1  -1  



/Users/fairliereese/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [254]:
print(df.dtypes)
sort_cols = ['Chromosome', 'Strand', 'gene_id', 'Start', 'End']
df = df.sort_values(by=sort_cols)
df.reset_index(drop=True, inplace=True)


Chromosome    category
Start            int64
End              int64
Strand        category
gene_id         object
source          object
dtype: object


In [255]:
ctrl = bepis()
print(ctrl.dtypes)
ctrl = ctrl.sort_values(by=sort_cols)
ctrl.reset_index(drop=True, inplace=True)

Start            int64
Chromosome    category
End              int64
gene_id         object
Strand        category
source          object
dtype: object


In [256]:
pd.testing.assert_frame_equal(ctrl, df, check_like=True)

In [245]:
# keeping entries from the old end df

# if we have strand information, enforce that
if 'Strand' in bed.columns:
    temp = df.join(bed,
                   strandedness='same',
                   suffix='_new',
                   slack=buffer,
                   how='left')
else:
    temp = df.join(bed,
                   strandedness=False,
                   suffix='_new', 
                   slack=buffer,
                   how='left')

# convert back to df
temp = temp.df
print(temp.head())

# if we have gene id information, also enforce that
if 'gene_id' in bed.columns:
    temp = temp.loc[(temp.gene_id == temp.gene_id_new)|(temp.gene_id_new == '-1')]

# reconcile sources
inds = temp.loc[temp.gene_id_new != '-1'].index
new_sources = temp.loc[inds, 'source']+','+temp.loc[inds, 'source_new']
temp.loc[inds, 'source'] = new_sources

# if we're adding in new ends from here
if add:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-245-2533ef766177>, line 32)

,Chromosome,Start,End,Name,Score,Strand,gene_id,tss,source,Start_new,End_new,Name_new,Score_new,Strand_new,gene_id_new,tss_new,source_new
0,chr11,50300939,50301040,ENSMUSG00000020368_2,.,-,ENSMUSG00000020368,2,"v1,v2",50300939,50301040,ENSMUSG00000020368_1,.,-,ENSMUSG00000020368,1,v2
1,chr11,50298268,50298369,ENSMUSG00000020368_3,.,-,ENSMUSG00000020368,3,v1,-1,-1,-1,-1,-,-1,-1,-1
2,chr11,50325622,50325723,ENSMUSG00000020368_1,.,-,ENSMUSG00000020368,1,v1,-1,-1,-1,-1,-,-1,-1,-1


In [ ]:
# adding entries from the new df
if 'Strand' in bed.columns:
    temp2 = df.join(bed,
                     strandedness='same',
                     suffix
                     

In [3]:
# # first subset the human ccres for those that are promoters 
# # and are w/i genomic region of Canx
# #  https://api.wenglab.org/screen_v13/fdownloads/V3/GRCh38-cCREs.bed
# fname = '/Users/fairliereese/mortazavi_lab/ref/ccre_hg38/GRCh38-cCREs.bed'
# df = pr.read_bed(fname)
# df = df['chr11', 50293961:50325673]

# # ok so the toy gtf I have is definitely mouse oops
# # https://api.wenglab.org/screen_v13/fdownloads/V3/mm10-ccREs.bed
# fname = '/Users/fairliereese/mortazavi_lab/ref/ccre_vM3/mm10-ccREs.bed'
# df = pr.read_bed(fname)
# df = df['chr11', 50293961:50325673].df
# df = df.loc[df.Strand.str.contains('pELS')] 
# df.to_csv('tests/files/Canx_ccre_pels.bed', sep='\t', index=False, header=None)

In [74]:
import pyranges

starts = [1, 10, 20]
ends = [15, 12, 25]
chroms = [1, 1, 1]
gids = ['gene_1', 'gene_2', 'gene_2']
names = ['a', 'b', 'c']
gr = pyranges.PyRanges(chromosomes=chroms, starts=starts, ends=ends).as_df()
gr['gene_id'] = gids
gr['Name'] = names
gr = pyranges.PyRanges(df=gr)

starts = [11, 11]
ends = [11, 11]
chroms = [1, 1]
names = ['d', 'e']
gids = ['gene_1', 'gene_2']
gr2 = pyranges.PyRanges(chromosomes=chroms, starts=starts, ends=ends).as_df()
gr2['gene_id'] = gids
gr2['Name'] = names
gr2 = pyranges.PyRanges(df=gr2)

temp = gr2.join(gr, how='left', strandedness=None).df
print(temp)
temp = temp.loc[temp.gene_id == temp.gene_id_b]

  Chromosome  Start  End gene_id Name  Start_b  End_b gene_id_b Name_b
0          1     11   11  gene_1    d        1     15    gene_1      a
1          1     11   11  gene_1    d       10     12    gene_2      b
2          1     11   11  gene_2    e        1     15    gene_1      a
3          1     11   11  gene_2    e       10     12    gene_2      b


In [75]:
temp

,Chromosome,Start,End,gene_id,Name,Start_b,End_b,gene_id_b,Name_b
0,1,11,11,gene_1,d,1,15,gene_1,a
3,1,11,11,gene_2,e,10,12,gene_2,b
